# Récursivité et optimisation

Est-ce que la récursivité a encore une mauvaise image ? À cause du coût à l'exécution ? Pourtant, en contre-partie, elle permet une expression des algorithmes qui montre leurs structures/propriétés logiques (la récursivité, c'est se baser sur des invariants d'algorithmes) et les optimisations permises par les compilateurs peuvent rendre les programmes aussi efficaces que les programmes itératifs.

Pour étudier les aspects "performances", prenons l'exemple du calcul des sommes d'entiers de 1 à N.

## Première approche, programme récursif initial, compilation normale

Le programme récursif le plus évident est le suivant :

In [2]:
%%writefile somme.c
#include <stdio.h>

int somme(int n) {
if (!n) {return 0;}
else {return n+somme(n-1);}}

int main (int argc, char * argv[]) {
int i;
if (argc==1)    {i=20;}
else {i=atoi(argv[1]);}
printf("1+2+...+%d=%d\n",i,somme(i));
return 0;}

Writing somme.c


Une compilation normale en ARM donne :  
(on peut reconnaitre le programme princpal (main),   
l'appel à somme et dans la fonction somme l'appel récursif à somme)

In [2]:
%%sh
arm-linux-gnueabi-gcc -S -static somme.c
cat somme.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"somme.c"
	.text
	.align	2
	.global	somme
	.type	somme, %function
somme:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #8
	str	r0, [fp, #-8]
	ldr	r3, [fp, #-8]
	cmp	r3, #0
	bne	.L2
	mov	r3, #0
	b	.L3
.L2:
	ldr	r3, [fp, #-8]
	sub	r3, r3, #1
	mov	r0, r3
	bl	somme
	mov	r2, r0
	ldr	r3, [fp, #-8]
	add	r3, r2, r3
.L3:
	mov	r0, r3
	sub	sp, fp, #4
	ldmfd	sp!, {fp, pc}
	.size	somme, .-somme
	.section	.rodata
	.align	2
.LC0:
	.ascii	"1+2+...+%d=%d\012\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 16
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #16
	str	r0, [fp, #-16]
	str	r

À l'exécution, pour de petites valeurs c'est ok. 

In [3]:
%%sh
arm-linux-gnueabi-gcc -static somme.c
qemu-arm a.out

1+2+...+20=210


Par contre, avant même que cela ne devienne un peu long à calculer, des limites s'expriment (au dela de 500 000) :  
la faute à la pile d'appel !

In [8]:
%%sh
arm-linux-gnueabi-gcc -static somme.c
qemu-arm a.out 500000

1+2+...+500000=446198416


In [9]:
%%sh
arm-linux-gnueabi-gcc -static somme.c
qemu-arm a.out 600000

qemu: uncaught target signal 11 (Segmentation fault) - core dumped
Segmentation fault (core dumped)


N.B : par ailleurs, les résultats sont faux (la valeur attendue est  $\frac{n(n+1)}{2}$), ce qui pour 500 000 devrait donner qlq chose de l'ordre de 125 000 000 000). On aurait pu s'en rendre compte, si par hasard on avait demandé (par exemple) :  
Quoi, un résultat négatif ?

In [6]:
%%sh
arm-linux-gnueabi-gcc -static somme.c
qemu-arm a.out 400000

1+2+...+400000=-1604178624


## Amélioration du programme

La récursivité ne demande pas toujours une pile d'appel ! Dans le cas d'une récursivité "terminale" (ou il n'y que des appels récursifs à la fin de la fonction), la pile d'appel n'est pas nécessaire.

En introduisant un accumulateur, la fonction suivante est récursive terminale :

In [10]:
%%writefile sommeTal.c
#include <stdio.h>

int somme(int a,int s) {
if (!a) {return s;}
else {return somme(a-1,a+s);}}

int main (int argc, char *argv[]) {
int i;
if (argc==1) {i=20;}
else {i=atoi(argv[1]);}
printf("1+2+...+%d=%d\n",i,somme(i,0));
return 0;}

Writing sommeTal.c


Mais cela ne change pas grand chose (le compilateur fait une traduction simple) :

In [8]:
%%sh
arm-linux-gnueabi-gcc -S -static sommeTal.c
cat sommeTal.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"sommeTal.c"
	.text
	.align	2
	.global	somme
	.type	somme, %function
somme:
	@ args = 0, pretend = 0, frame = 8
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #8
	str	r0, [fp, #-8]
	str	r1, [fp, #-12]
	ldr	r3, [fp, #-8]
	cmp	r3, #0
	bne	.L2
	ldr	r3, [fp, #-12]
	b	.L3
.L2:
	ldr	r3, [fp, #-8]
	sub	r2, r3, #1
	ldr	r1, [fp, #-8]
	ldr	r3, [fp, #-12]
	add	r3, r1, r3
	mov	r0, r2
	mov	r1, r3
	bl	somme
	mov	r3, r0
.L3:
	mov	r0, r3
	sub	sp, fp, #4
	ldmfd	sp!, {fp, pc}
	.size	somme, .-somme
	.section	.rodata
	.align	2
.LC0:
	.ascii	"1+2+...+%d=%d\012\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 16
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, 

In [15]:
%%sh
arm-linux-gnueabi-gcc -static sommeTal.c
qemu-arm a.out 500000

1+2+...+500000=446198416


Le résultat est le même que précédement (faux, cf. plus haut), et cela plante au même endroit :

In [10]:
%%sh
arm-linux-gnueabi-gcc -static sommeTal.c
qemu-arm a.out 600000

qemu: uncaught target signal 11 (Segmentation fault) - core dumped
Segmentation fault (core dumped)


N.B. : c'est assez étonnant, en fait, on aurait pu imaginé que la pile se remplisse plus vite (la fonction a 2 arguments au lieu d'1 !). En regardant le code compilé, c'est peut-être plus clair ? Non, il y a autant de place prise sur la pile. Pourquoi pas plus ? C'est peut-être, que le code avant, avait, en fait besoin de plus de place que seulement le nombre de paramètres (!) par exemple pour faire la calcul final (1 temporaire ?).  
Si artificiellement, on augmentre le nombre de paramètre sur la pile pour une version ad'hoc récursive terminale, on obtient bien une fonction qui remplie plus vite la pile :

In [16]:
%%writefile sommeAdHoc.c
#include <stdio.h>

int somme(int a,int s, int t) {
if (!a) {return s;}
else {return somme(a-1,a+s, t);}}

int main (int argc, char *argv[]) {
int i;
if (argc==1) {i=20;}
else {i=atoi(argv[1]);}
printf("1+2+...+%d=%d\n",i,somme(i,0,0));
return 0;}

Writing sommeAdHoc.c


In [12]:
%%sh
arm-linux-gnueabi-gcc -S -static sommeAdHoc.c
cat sommeAdHoc.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 6
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"sommeAdHoc.c"
	.text
	.align	2
	.global	somme
	.type	somme, %function
somme:
	@ args = 0, pretend = 0, frame = 16
	@ frame_needed = 1, uses_anonymous_args = 0
	stmfd	sp!, {fp, lr}
	add	fp, sp, #4
	sub	sp, sp, #16
	str	r0, [fp, #-8]
	str	r1, [fp, #-12]
	str	r2, [fp, #-16]
	ldr	r3, [fp, #-8]
	cmp	r3, #0
	bne	.L2
	ldr	r3, [fp, #-12]
	b	.L3
.L2:
	ldr	r3, [fp, #-8]
	sub	r2, r3, #1
	ldr	r1, [fp, #-8]
	ldr	r3, [fp, #-12]
	add	r3, r1, r3
	mov	r0, r2
	mov	r1, r3
	ldr	r2, [fp, #-16]
	bl	somme
	mov	r3, r0
.L3:
	mov	r0, r3
	sub	sp, fp, #4
	ldmfd	sp!, {fp, pc}
	.size	somme, .-somme
	.section	.rodata
	.align	2
.LC0:
	.ascii	"1+2+...+%d=%d\012\000"
	.text
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 16
	@ frame_needed = 

In [20]:
%%sh
arm-linux-gnueabi-gcc -static sommeAdHoc.c
qemu-arm a.out 300000

1+2+...+300000=2050477040


## Et si le compilateur aidait ?

On repart du programme simple de départ et on demande au compilateur s'il peut aider.

In [14]:
%%writefile somme.c
#include <stdio.h>

int somme(int n) {
if (!n) {return 0;}
else {return n+somme(n-1);}}

int main (int argc, char * argv[]) {
int i;
if (argc==1)    {i=20;}
else {i=atoi(argv[1]);}
printf("1+2+...+%d=%d\n",i,somme(i));
return 0;}

Overwriting somme.c


In [21]:
%%sh
arm-linux-gnueabi-gcc -O -S -static somme.c
cat somme.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 1
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"somme.c"
	.text
	.align	2
	.global	somme
	.type	somme, %function
somme:
	@ args = 0, pretend = 0, frame = 0
	@ frame_needed = 0, uses_anonymous_args = 0
	stmfd	sp!, {r4, lr}
	subs	r4, r0, #0
	beq	.L3
	sub	r0, r4, #1
	bl	somme
	add	r0, r0, r4
	ldmfd	sp!, {r4, pc}
.L3:
	mov	r0, #0
	ldmfd	sp!, {r4, pc}
	.size	somme, .-somme
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 0
	@ frame_needed = 0, uses_anonymous_args = 0
	stmfd	sp!, {r4, lr}
	cmp	r0, #1
	moveq	r4, #20
	beq	.L5
	ldr	r0, [r1, #4]
	bl	atoi
	mov	r4, r0
	b	.L5
.L5:
	mov	r0, r4
	bl	somme
	mov	r3, r0
	mov	r0, #1
	ldr	r1, .L7
	mov	r2, r4
	bl	__printf_chk
	mov	r0, #0
	ldmfd	sp!, {r4, pc}
.L8:
	.align	2
.L7:
	.word	.LC0
	.size	main, .-main
	.section	.rodata.s

Effectivement, il y a eu quelques optimisations, la pile est moins sollicitées, mais cela ne va pas suffire et on peut imaginer que cela va juste planter 2 fois plus loin (entre 1 000 0000 et 1 200 000) :

In [28]:
%%sh
arm-linux-gnueabi-gcc -O -static somme.c
qemu-arm a.out 6000000

qemu: uncaught target signal 11 (Segmentation fault) - core dumped
Segmentation fault (core dumped)


In [34]:
%%sh
arm-linux-gnueabi-gcc -O2 -static somme.c
qemu-arm a.out 1000000000

1+2+...+1000000000=-243309312


Cependant, le compilateur peut faire plus (beaucoup plus), il peut même essayer de reconnaitre une forme de récursivité qui se ramène (facilement) à une récursivité terminale et traduire mieux (il y a quelques années, il était un peu moins fort, il fallait que la récursité terminale soit explicite dès le départ ; et/ou il fallait lui demander explicitement de rechercher des récursivités terminales) :  
(en particulier, regarder la fonction "main", il n'y a plus de référence à "somme", en fait la fonction est inline, on reconnait la boucle, le +n et le -1 en .L9 au lieu d'une récursivité)

In [29]:
%%sh
arm-linux-gnueabi-gcc -O2 -S -static somme.c
cat somme.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 2
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"somme.c"
	.text
	.align	2
	.global	somme
	.type	somme, %function
somme:
	@ args = 0, pretend = 0, frame = 0
	@ frame_needed = 0, uses_anonymous_args = 0
	@ link register save eliminated.
	subs	r3, r0, #0
	beq	.L4
	mov	r0, #0
.L3:
	add	r0, r0, r3
	subs	r3, r3, #1
	bne	.L3
	bx	lr
.L4:
	mov	r0, r3
	bx	lr
	.size	somme, .-somme
	.section	.text.startup,"ax",%progbits
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 0
	@ frame_needed = 0, uses_anonymous_args = 0
	cmp	r0, #1
	stmfd	sp!, {r3, lr}
	beq	.L14
	ldr	r0, [r1, #4]
	bl	atoi
	subs	r2, r0, #0
	beq	.L15
.L11:
	mov	r1, r2
	mov	r3, #0
.L9:
	add	r3, r3, r1
	subs	r1, r1, #1
	bne	.L9
.L10:
	ldr	r1, .L16
	mov	r0, #1
	bl	__printf_chk
	mov	r0, #0
	ldmfd	sp!, {r3, pc}
.L1

## Conclusion

* Avant optimisation : un programme limité
* Après optimisation : une récursivité transformée en boucle (mais un programme qui continue de donner des résultats faux quand cela dépasse le milliard (32 bits))

N.B. : pour de petites valeurs, l'optimisation peut même changer radicalement le programme (cas où le programme n'a pas d'entrées et fonctionne avec des constantes) :  
Regarder le "main" de l'exécutable, le résultat du calcul est mis dès le départ !

In [19]:
%%writefile sommeCst.c
#include <stdio.h>

int somme(int n) {
if (!n) {return 0;}
else {return n+somme(n-1);}}

int main (int argc, char * argv[]) {
int i=20;
printf("1+2+...+%d=%d\n",i,somme(i));
return 0;}

Overwriting sommeCst.c


In [20]:
%%sh
arm-linux-gnueabi-gcc -O2 -S -static sommeCst.c
cat sommeCst.s

	.arch armv5t
	.fpu softvfp
	.eabi_attribute 20, 1
	.eabi_attribute 21, 1
	.eabi_attribute 23, 3
	.eabi_attribute 24, 1
	.eabi_attribute 25, 1
	.eabi_attribute 26, 2
	.eabi_attribute 30, 2
	.eabi_attribute 34, 0
	.eabi_attribute 18, 4
	.file	"sommeCst.c"
	.text
	.align	2
	.global	somme
	.type	somme, %function
somme:
	@ args = 0, pretend = 0, frame = 0
	@ frame_needed = 0, uses_anonymous_args = 0
	@ link register save eliminated.
	subs	r3, r0, #0
	beq	.L4
	mov	r0, #0
.L3:
	add	r0, r0, r3
	subs	r3, r3, #1
	bne	.L3
	bx	lr
.L4:
	mov	r0, r3
	bx	lr
	.size	somme, .-somme
	.section	.text.startup,"ax",%progbits
	.align	2
	.global	main
	.type	main, %function
main:
	@ args = 0, pretend = 0, frame = 0
	@ frame_needed = 0, uses_anonymous_args = 0
	stmfd	sp!, {r3, lr}
	ldr	r1, .L8
	mov	r2, #20
	mov	r3, #210
	mov	r0, #1
	bl	__printf_chk
	mov	r0, #0
	ldmfd	sp!, {r3, pc}
.L9:
	.align	2
.L8:
	.word	.LC0
	.size	main, .-main
	.section	.rodata.str1.4,"aMS",%progbits,1
	.align	2
.LC0:
	.ascii	"1+2+...+%d=%d

## et en X86 ?

C'est pareil ! (les erreurs sont juste sur des paliers différents)

In [21]:
%%writefile somme.c
#include <stdio.h>
#include <stdlib.h>

int somme(int n) {
if (!n) {return 0;}
else {return n+somme(n-1);}}

int main (int argc, char * argv[]) {
int i;
if (argc==1)    {i=20;}
else {i=atoi(argv[1]);}
printf("1+2+...+%d=%d\n",i,somme(i));
return 0;}

Overwriting somme.c


In [22]:
%%sh
gcc -S somme.c
cat somme.s

	.file	"somme.c"
	.text
	.globl	somme
	.type	somme, @function
somme:
.LFB2:
	.cfi_startproc
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	subq	$16, %rsp
	movl	%edi, -4(%rbp)
	cmpl	$0, -4(%rbp)
	jne	.L2
	movl	$0, %eax
	jmp	.L3
.L2:
	movl	-4(%rbp), %eax
	subl	$1, %eax
	movl	%eax, %edi
	call	somme
	movl	-4(%rbp), %edx
	addl	%edx, %eax
.L3:
	leave
	.cfi_def_cfa 7, 8
	ret
	.cfi_endproc
.LFE2:
	.size	somme, .-somme
	.section	.rodata
.LC0:
	.string	"1+2+...+%d=%d\n"
	.text
	.globl	main
	.type	main, @function
main:
.LFB3:
	.cfi_startproc
	pushq	%rbp
	.cfi_def_cfa_offset 16
	.cfi_offset 6, -16
	movq	%rsp, %rbp
	.cfi_def_cfa_register 6
	subq	$32, %rsp
	movl	%edi, -20(%rbp)
	movq	%rsi, -32(%rbp)
	cmpl	$1, -20(%rbp)
	jne	.L5
	movl	$20, -4(%rbp)
	jmp	.L6
.L5:
	movq	-32(%rbp), %rax
	addq	$8, %rax
	movq	(%rax), %rax
	movq	%rax, %rdi
	call	atoi
	movl	%eax, -4(%rbp)
.L6:
	movl	-4(%rbp), %eax
	movl	%eax, %edi
	call	somme
	movl	%eax, %edx
	movl	-4(%rbp

In [23]:
%%sh
gcc  somme.c
./a.out

1+2+...+20=210


In [24]:
%%sh
gcc  somme.c
./a.out 200000

1+2+...+200000=-1474736480


In [25]:
%%sh
gcc  somme.c
./a.out 300000

Segmentation fault (core dumped)


In [26]:
%%sh
gcc  -O somme.c
./a.out 300000

1+2+...+300000=2050477040


In [27]:
%%sh
gcc  -O somme.c
./a.out 600000

Segmentation fault (core dumped)


In [28]:
%%sh
gcc  -O2 somme.c
./a.out 1000000000

1+2+...+1000000000=-243309312


In [29]:
%%sh
gcc -O2 -S somme.c
cat somme.s

	.file	"somme.c"
	.text
	.p2align 4,,15
	.globl	somme
	.type	somme, @function
somme:
.LFB39:
	.cfi_startproc
	xorl	%eax, %eax
	testl	%edi, %edi
	je	.L2
	.p2align 4,,10
	.p2align 3
.L3:
	addl	%edi, %eax
	subl	$1, %edi
	jne	.L3
.L2:
	rep ret
	.cfi_endproc
.LFE39:
	.size	somme, .-somme
	.section	.rodata.str1.1,"aMS",@progbits,1
.LC0:
	.string	"1+2+...+%d=%d\n"
	.section	.text.startup,"ax",@progbits
	.p2align 4,,15
	.globl	main
	.type	main, @function
main:
.LFB40:
	.cfi_startproc
	subq	$8, %rsp
	.cfi_def_cfa_offset 16
	cmpl	$1, %edi
	je	.L19
	movq	8(%rsi), %rdi
	movl	$10, %edx
	xorl	%esi, %esi
	call	strtol
	testl	%eax, %eax
	movl	%eax, %edx
	je	.L20
.L13:
	movl	%edx, %eax
	xorl	%ecx, %ecx
	.p2align 4,,10
	.p2align 3
.L11:
	addl	%eax, %ecx
	subl	$1, %eax
	jne	.L11
.L12:
	movl	$.LC0, %esi
	movl	$1, %edi
	xorl	%eax, %eax
	call	__printf_chk
	xorl	%eax, %eax
	addq	$8, %rsp
	.cfi_remember_state
	.cfi_def_cfa_offset 8
	ret
.L19:
	.cfi_restore_state
	movl	$20, %edx
	jmp	.L13
.L20:
	xorl	%ecx, %ecx

## Et avec un autre compilateur ? (clang)

In [34]:
%%sh
clang -S somme.c
cat somme.s

	.file	"somme.c"
	.text
	.globl	somme
	.align	16, 0x90
	.type	somme,@function
somme:                                  # @somme
	.cfi_startproc
# BB#0:
	pushq	%rbp
.Ltmp2:
	.cfi_def_cfa_offset 16
.Ltmp3:
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
.Ltmp4:
	.cfi_def_cfa_register %rbp
	subq	$16, %rsp
	movl	%edi, -8(%rbp)
	cmpl	$0, -8(%rbp)
	jne	.LBB0_2
# BB#1:
	movl	$0, -4(%rbp)
	jmp	.LBB0_3
.LBB0_2:
	movl	-8(%rbp), %eax
	movl	-8(%rbp), %ecx
	subl	$1, %ecx
	movl	%ecx, %edi
	movl	%eax, -12(%rbp)         # 4-byte Spill
	callq	somme
	movl	-12(%rbp), %ecx         # 4-byte Reload
	addl	%eax, %ecx
	movl	%ecx, -4(%rbp)
.LBB0_3:
	movl	-4(%rbp), %eax
	addq	$16, %rsp
	popq	%rbp
	ret
.Ltmp5:
	.size	somme, .Ltmp5-somme
	.cfi_endproc

	.globl	main
	.align	16, 0x90
	.type	main,@function
main:                                   # @main
	.cfi_startproc
# BB#0:
	pushq	%rbp
.Ltmp8:
	.cfi_def_cfa_offset 16
.Ltmp9:
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
.Ltmp10:
	.cfi_def_cfa_register %rbp
	subq	$32, %rsp
	movl	

In [30]:
%%sh
clang somme.c
./a.out 200000

1+2+...+200000=-1474736480


In [31]:
%%sh
clang somme.c
./a.out 300000

Segmentation fault (core dumped)


In [32]:
%%sh
clang -O somme.c
./a.out 1000000000

1+2+...+1000000000=-243309312


In [33]:
%%sh
clang -O -S somme.c
cat somme.s

	.file	"somme.c"
	.text
	.globl	somme
	.align	16, 0x90
	.type	somme,@function
somme:                                  # @somme
	.cfi_startproc
# BB#0:
                                        # kill: EDI<def> EDI<kill> RDI<def>
	xorl	%eax, %eax
	testl	%edi, %edi
	je	.LBB0_2
# BB#1:                                 # %.lr.ph
	leal	-1(%rdi), %eax
	leal	-2(%rdi), %ecx
	imulq	%rax, %rcx
                                        # kill: EAX<def> EAX<kill> RAX<kill>
	imull	%eax, %eax
	shrq	%rcx
	addl	%edi, %eax
	subl	%ecx, %eax
.LBB0_2:
	ret
.Ltmp0:
	.size	somme, .Ltmp0-somme
	.cfi_endproc

	.globl	main
	.align	16, 0x90
	.type	main,@function
main:                                   # @main
	.cfi_startproc
# BB#0:
	pushq	%rbx
.Ltmp3:
	.cfi_def_cfa_offset 16
.Ltmp4:
	.cfi_offset %rbx, -16
	cmpl	$1, %edi
	movl	$20, %eax
	je	.LBB1_2
# BB#1:
	movq	8(%rsi), %rdi
	xorl	%ebx, %ebx
	xorl	%esi, %esi
	movl	$10, %edx
	callq	strtol
	testl	%eax, %eax
	movl	$0, %edx
	je	.LBB1_3
.LBB1_2:                         